In [1]:
import os, urllib
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

In [2]:
# This is what makes the magic functions work:

%load_ext sql

# %%sql to create a multi-line sql block that outputs a dataframe
# %sql to execute a single line of sql that can go straight into a python variable.
# Be sure to install the ipython-sql for this to work. 

### Create the connection string

The obdc driver must be installed to make this work. The credentials reference a windows virtual machine running within a linux host computer.

In [3]:
# View the drivers installed on the system:

pyodbc.drivers()


['SQL Server',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

The connection string is established in the first sql magic function.

In [4]:
%%sql mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server

SELECT DISTINCT TOP 5 cause
FROM causes
ORDER BY cause ASC;

Done.


cause
297
298
302
321
322


### Select the top 10

In [35]:
%%sql 

SELECT TOP 10 * 
FROM causes

#diff

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,51,1,28,520,1,2010,0.0,0.0,0.0
1,51,2,28,520,1,2010,0.0,0.0,0.0
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191


### Use GROUP BY to aggregate data

In [4]:
%%sql

SELECT sex, sum(val)
FROM causes
GROUP BY sex

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


sex,
3,540531575.3892027
1,3343289734.615242
2,2830323502.7642317


### Filter using WHERE and a CONDITIONAL

In [9]:
%%sql

SELECT TOP 10 location, val
FROM causes
WHERE val < 10 AND val > 8
ORDER BY location DESC

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


location,val
522,9.837203979492188
522,9.138937950134277
522,8.108078002929688
522,8.955031394958496
522,8.321249008178711
522,9.00266170501709
522,9.531730651855469
522,9.212886810302734
522,8.557770729064941
522,9.661809921264648


### Perform a LEFT JOIN on multiple fields

In [10]:
%%sql

SELECT TOP 10 cau.*, cause_name, age_group_name, sex_label
FROM causes cau
LEFT JOIN cause_id cid ON cid.cause_id = cau.cause
LEFT JOIN age_id age ON age.age_group_id = cau.age
LEFT JOIN sex_id sex ON sex.sex_id = cau.sex

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower,cause_name,age_group_name,sex_label
1,51,1,28,520,1,2010,0.0,0.0,0.0,Other chronic respiratory diseases,<1 year,Male
1,51,2,28,520,1,2010,0.0,0.0,0.0,Other chronic respiratory diseases,<1 year,Female
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386,Other chronic respiratory diseases,80 to 84,Male
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373,Other chronic respiratory diseases,80 to 84,Female
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916,Other chronic respiratory diseases,85 to 89,Male
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389,Other chronic respiratory diseases,85 to 89,Female
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836,Other chronic respiratory diseases,90 to 94,Male
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085,Other chronic respiratory diseases,90 to 94,Female
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844,Other chronic respiratory diseases,20 plus,Male
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191,Other chronic respiratory diseases,20 plus,Female


### Use BETWEEN to limit the number of CONDITIONAL statements

In [11]:
%%sql

SELECT TOP 10 cid.cause_name, sum(val) AS val
FROM causes AS cau
LEFT JOIN cause_id AS cid ON cid.cause_id = cau.cause
LEFT JOIN age_id AS age ON age.age_group_id = cau.age
LEFT JOIN sex_id AS sex ON sex.sex_id = cau.sex
WHERE cau.year = 2019 AND age.age_group_id BETWEEN 2 AND 21
GROUP BY cid.cause_name
ORDER BY val DESC


-- The 'AS' clauses are not necessary, but they make the code more readable
-- The BETWEEN statement is inclusive, so 2 and 21 are included in the query. 

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


cause_name,val
Ischemic heart disease,9132782.320356125
Stroke,6549283.146548615
Chronic obstructive pulmonary disease,3279443.3530463926
Lower respiratory infections,2491470.6013133274
"Tracheal, bronchus, and lung cancer",2041697.5392837853
Neonatal disorders,1881364.520554876
Alzheimer's disease and other dementias,1622425.9312281173
Diabetes mellitus,1549595.7159495763
Diarrheal diseases,1533773.2230857266
Cirrhosis and other chronic liver diseases,1471146.8246638603


### Use PARENTHESES in CONDITIONAL statements

In [46]:
%%sql

SELECT TOP 15 cau.*, cause_name, age_group_name, sex_label
FROM causes cau
LEFT JOIN cause_id cid ON cid.cause_id = cau.cause
LEFT JOIN age_id age ON age.age_group_id = cau.age
LEFT JOIN sex_id sex ON sex.sex_id = cau.sex
WHERE (cause = 520 OR cause = 3) AND age = 3

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower,cause_name,age_group_name,sex_label
1,67,1,3,520,1,2010,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Male
1,67,2,3,520,1,2010,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Female
1,83,1,3,520,1,2010,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Male
1,83,2,3,520,1,2010,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Female
1,144,1,3,520,1,2011,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Male
1,144,2,3,520,1,2011,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Female
1,99,1,3,520,1,2010,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Male
1,99,2,3,520,1,2010,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Female
1,160,1,3,520,1,2011,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Male
1,160,2,3,520,1,2011,0.0,0.0,0.0,Other chronic respiratory diseases,Late Neonatal,Female


In this case, the parentheses change the logic so that the OR operator is evaluated before the AND operator. Without them, AND would be evaluated before OR, in effect meaning that cause = 3 and age = 3 would be grouped together, and cause = 520 would be evaluated alone. 

<a href='https://learn.microsoft.com/en-us/sql/t-sql/language-elements/operator-precedence-transact-sql?view=sql-server-ver16'>Here</a> is the order of operations for T-SQL.

### Use an ALIAS column

In [ ]:
%%sql

SELECT AVG(val) AS average
FROM causes



 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


average
245.57375339048713


### Basic Aggregate Functions

In [39]:
%%sql

SELECT 
COUNT(val) as count, 
SUM(val) AS sum, 
MIN(val) as min, 
MAX(val) as max,
AVG(val) AS average, 
STDEV(val) as stdev
FROM causes 



 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


count,sum,min,max,average,stdev
27340645,6714144812.766855,0.0,2189175.011909225,245.57375339048713,5441.580775017772


### Create a VIEW

In [40]:
%%sql

CREATE VIEW temp_view
AS
SELECT TOP 10 *, ROW_NUMBER() OVER (ORDER BY measure) AS RN
FROM causes;
SELECT * FROM temp_view;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower,RN
1,51,1,28,520,1,2010,0.0,0.0,0.0,1
1,51,2,28,520,1,2010,0.0,0.0,0.0,2
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386,3
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373,4
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916,5
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389,6
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836,7
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085,8
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844,9
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191,10


### Alter a VIEW

This code replaces the view with the new selection.

In [35]:
%%sql

ALTER VIEW temp_view
AS
SELECT TOP 15 *, ROW_NUMBER() OVER (ORDER BY measure) AS RN
FROM causes;
SELECT * FROM temp_view

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower,RN
1,51,1,28,520,1,2010,0.0,0.0,0.0,1
1,51,2,28,520,1,2010,0.0,0.0,0.0,2
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386,3
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373,4
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916,5
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389,6
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836,7
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085,8
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844,9
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191,10


### DROP a VIEW

In [41]:
%%sql

DROP VIEW temp_view

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


[]

### CREATE a TABLE using SQL code

In [11]:
%%sql

CREATE TABLE temp_table 
(
	column1 varchar(10),
	column2 varchar(10),
	column3 float
)

-- column names are followed by data types

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


[]

### INSERT VALUES INTO TABLE

In [12]:
%%sql

INSERT INTO temp_table (column1, column2, column3)
VALUES 
	('a','b', 1.1),
    ('c','d', 2.2),
    ('e','f', 3.3)

-- make sure that the text has quotes

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
3 rows affected.


[]

View the data.

In [13]:
%%sql

SELECT * FROM temp_table

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


column1,column2,column3
a,b,1.1
c,d,2.2
e,f,3.3


### CREATE a TABLE from an existing TABLE

In [27]:
%%sql

SELECT TOP 10 *
INTO temporary_table
FROM causes;
SELECT * FROM temporary_table

-- #diff

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
10 rows affected.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,51,1,28,520,1,2010,0.0,0.0,0.0
1,51,2,28,520,1,2010,0.0,0.0,0.0
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191


### INSERT values from one table INTO another

In [28]:
%%sql

INSERT INTO temporary_table
SELECT TOP 10 * FROM causes;
SELECT * FROM temporary_table;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
10 rows affected.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,51,1,28,520,1,2010,0.0,0.0,0.0
1,51,2,28,520,1,2010,0.0,0.0,0.0
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191


### CREATE COLUMNS using ALTER TABLE

In [29]:
%%sql

ALTER TABLE temporary_table
ADD rounded float, 
    times3 float, 
    over2 float;
SELECT * FROM temporary_table;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower,rounded,times3,over2
1,51,1,28,520,1,2010,0.0,0.0,0.0,None,None,None
1,51,2,28,520,1,2010,0.0,0.0,0.0,None,None,None
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386,None,None,None
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373,None,None,None
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916,None,None,None
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389,None,None,None
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836,None,None,None
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085,None,None,None
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844,None,None,None
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191,None,None,None


### Add values to COLUMNS using UPDATE and SET

In [30]:
%%sql

UPDATE temporary_table
SET rounded = ROUND(VAL,2), 
	times3 = val*3, 
	over2 = val/2;
SELECT * FROM temporary_table;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
20 rows affected.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower,rounded,times3,over2
1,51,1,28,520,1,2010,0.0,0.0,0.0,0.0,0.0,0.0
1,51,2,28,520,1,2010,0.0,0.0,0.0,0.0,0.0,0.0
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386,1.8,5.39522123336792,0.8992035388946533
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373,2.76,8.267490148544312,1.3779150247573853
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916,1.27,3.8213850259780884,0.6368975043296814
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389,2.87,8.60483694076538,1.4341394901275635
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836,1.02,3.0479904413223267,0.5079984068870544
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085,1.83,5.484657526016235,0.9141095876693726
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844,27.65,82.96147155761719,13.826911926269531
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191,22.19,66.57551765441895,11.095919609069824


### DROP COLUMNS

In [31]:
%%sql

ALTER TABLE temporary_table
DROP COLUMN rounded, 
            times3, 
            over2;
SELECT * FROM temporary_table;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,51,1,28,520,1,2010,0.0,0.0,0.0
1,51,2,28,520,1,2010,0.0,0.0,0.0
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191


### Replace values using UPDATE and SET

In [83]:
%%sql

UPDATE temporary_table
SET lower = NULL
WHERE lower = 0;
SELECT * FROM temporary_table;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
4 rows affected.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,51,1,28,520,1,2010,0.0,0.0,None
1,51,2,28,520,1,2010,0.0,0.0,None
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191


### DELETE table values

In [87]:
%%sql

DELETE FROM temporary_table
WHERE lower = 0;
SELECT * FROM temporary_table;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
4 rows affected.
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373


### Get all TABLE names in DATABASE

In [33]:
%%sql

SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='GBD'

-- #diff

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


TABLE_NAME
cause_hier
causes
sex_id
pop
temp_table
temporary_table
code
cause_id
age_id
metric_id


### DROP TABLE

In [91]:
%%sql

DROP TABLE temp_table;
DROP TABLE temporary_table;
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='GBD'

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.
Done.
Done.


TABLE_NAME
cause_hier
causes
sex_id
pop
code
cause_id
age_id
metric_id
location_id
location_hier


### Use LIKE to find numbers that are approximate to the given value

In [92]:
%%sql

SELECT *
FROM causes
WHERE val LIKE 345

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,168,2,188,297,1,2015,344.9999551362209,492.7785177634799,190.42534230667445
1,10,1,234,426,1,2010,345.0000606585086,513.0531273621685,259.9976643916508


### Subqueries

#### Using the WITH syntax

In [8]:
%%sql

WITH subquery AS (
    SELECT year, cause, val, location
    FROM causes
    WHERE sex = 3 AND Age = 22
)
SELECT TOP 10 * FROM subquery;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


year,cause,val,location
2010,958,0.0016089386474696982,305
2010,959,0.0011076247768448894,305
2010,380,0.5109784824635506,320
2010,387,0.05002098040241527,320
2010,391,0.0012289050661582366,320
2010,393,0.29658958035588406,320
2010,400,0.00701101968080948,320
2010,408,0.02197486138971005,320
2010,529,0.019817507665936006,320
2010,530,0.2571917438911256,320


#### Using the Inline Syntax

In [11]:
%%sql

SELECT TOP 10 * FROM
(
    SELECT year, cause, val, location
    FROM causes
    WHERE sex = 3 AND Age = 22
) AS subquery;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


year,cause,val,location
2010,958,0.0016089386474696982,305
2010,959,0.0011076247768448894,305
2010,380,0.5109784824635506,320
2010,387,0.05002098040241527,320
2010,391,0.0012289050661582366,320
2010,393,0.29658958035588406,320
2010,400,0.00701101968080948,320
2010,408,0.02197486138971005,320
2010,529,0.019817507665936006,320
2010,530,0.2571917438911256,320


Subqueries can also be saved as Views and recalled that way. A View is actually a saved query. Data is not copied in a View.

### Window Functions

This code shows the sum for all causes for each country ranked for every year, where age = ALL and sex = ALL. Essentially, the table contains values within the table that have already been aggregated. We are selecting those aggregations, and then aggregating even further to find the total fatality rate for each country for every year. The windowed function is used to rank and partition by year.

ROW_NUMBER() can also be used for this query. The main difference between the two is in how they handle ties.

In [39]:
%%sql

SELECT sub2.*, location_id.location_name FROM (
    SELECT year, location, val, RANK() OVER (PARTITION BY year ORDER BY val DESC) as ranks, sex, age
    FROM (
        SELECT year, location, sum(val) as val, sex, age
        FROM causes
        GROUP BY year, location, sex, age
        HAVING sex = 3 AND age = 22
    ) AS sub1
    ) AS sub2
LEFT JOIN location_id ON location_id.location_id = sub2.location
WHERE ranks <= 3
ORDER BY year ASC, val DESC


 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


year,location,val,ranks,sex,age,location_name
2010,6,9504937.448965307,1,3,22,China
2010,163,8640665.835880565,2,3,22,India
2010,102,2474653.8557731807,3,3,22,United States of America
2011,6,9583385.53701778,1,3,22,China
2011,163,8786057.626366593,2,3,22,India
2011,102,2519634.166600004,3,3,22,United States of America
2012,6,9579151.275563974,1,3,22,China
2012,163,8841396.790844485,2,3,22,India
2012,102,2551922.8854898005,3,3,22,United States of America
2013,6,9626095.221899824,1,3,22,China


Windowed Functions: 

- ROW_NUMBER()
- RANK()
- PERCENT_RANK()
- FIRST_VALUE()
- LAST_VALUE()
- DENSE_RANK()
- CUME_DIST()
- NTILE()
- NTH_VALUE()
- LAG()
- LEAD()

## Additional Differences between Standard SQL and T-SQL

https://learnsql.com/blog/14-differences-sql-vs-tsql/

### Naming Conventions

In T-SQL, brackets can be used to designate tables, views, and columns with spaces or special characters. Quotation marks can also be used as is done in standard SQL.

In [95]:
%%sql 

SELECT TOP 5 [Cause ID]
FROM cause_hier

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


Cause ID
294
295
955
298
948


### Returns

T-SQL can return values without a reference to a table.

In [96]:
%%sql

SELECT 27/9

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


""
3


### LIMITS

T-SQL, uses a different syntax to limit the number of rows displayed. Either 'TOP' can be used, as was exemplified earlier, or this syntax below. OFFSET is required for the query to run. It designates the number of rows to skip before displaying the results. ORDER BY is also required.

In [97]:
%%sql

SELECT *
FROM causes
ORDER BY cause OFFSET 0 ROWS FETCH FIRST 10 ROWS ONLY

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,522,1,1,297,1,2011,130.5975799560547,261.2641906738281,50.05995559692383
1,522,2,1,297,1,2011,83.42724609375,167.2034454345703,27.466636657714844
1,522,1,4,297,1,2011,47.079612731933594,105.88108825683594,15.849140167236328
1,522,2,4,297,1,2011,36.9808464050293,79.20865631103516,10.557940483093262
1,522,1,5,297,1,2011,83.5179672241211,179.57154846191406,30.637060165405273
1,522,2,5,297,1,2011,46.4463996887207,94.38202667236328,14.558396339416504
1,522,1,6,297,1,2011,22.57392692565918,44.70649719238281,8.853533744812012
1,522,2,6,297,1,2011,12.704204559326172,25.81943702697754,4.107003688812256
1,522,1,7,297,1,2011,9.837203979492188,19.064868927001953,3.829822063446045
1,522,2,7,297,1,2011,9.138937950134277,18.96198272705078,2.847052812576294


In [98]:
%%sql

-- equivalent syntax using top:

SELECT TOP 10 *
FROM causes

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


measure,location,sex,age,cause,metric,year,val,upper,lower
1,51,1,28,520,1,2010,0.0,0.0,0.0
1,51,2,28,520,1,2010,0.0,0.0,0.0
1,51,1,30,520,1,2010,1.7984070777893066,4.949845314025879,1.3105658292770386
1,51,2,30,520,1,2010,2.7558300495147705,6.83079719543457,1.7773444652557373
1,51,1,31,520,1,2010,1.2737950086593628,3.0925118923187256,0.9484407901763916
1,51,2,31,520,1,2010,2.868278980255127,7.565248966217041,1.8214061260223389
1,51,1,32,520,1,2010,1.0159968137741089,1.2909469604492188,0.6816396713256836
1,51,2,32,520,1,2010,1.8282191753387451,3.8931028842926025,1.143686056137085
1,51,1,37,520,1,2010,27.653823852539062,73.91763305664062,22.006675720214844
1,51,2,37,520,1,2010,22.19183921813965,56.39546203613281,15.032441139221191


### IDENTITY Column

SQL can be used to create columns that are automatically populated with values.

In [40]:
%%sql

--Standard SQL syntax

CREATE TABLE testTable (id DECIMAL GENERATED ALWAYS AS IDENTITY)

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
(pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near the keyword 'IDENTITY'. (156) (SQLExecDirectW)")
[SQL: --Standard SQL syntax

CREATE TABLE testTable (id DECIMAL GENERATED ALWAYS AS IDENTITY)]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [101]:
%%sql

--T-SQL syntax

CREATE TABLE testTable (id INTEGER IDENTITY, column1 float);
INSERT INTO testTable (column1) VALUES (1.1),(2.2),(3.3);
SELECT * FROM testTable;

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.
3 rows affected.
Done.


id,column1
1,1.1
2,2.2
3,3.3


In [100]:
%%sql
DROP TABLE testTable

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
Done.


[]

In [102]:
%%sql 

SELECT * FROM Tablecolumns

 * mssql+pyodbc://Windows-10\SQLEXPRESS/GBD?driver=SQL+Server
(pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'Tablecolumns'. (208) (SQLExecDirectW)")
[SQL: SELECT * FROM Tablecolumns]
(Background on this error at: https://sqlalche.me/e/14/f405)
